In [41]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 24, 8
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV, KFold, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.utils import resample

In [15]:
from numba import jit
import time
import gc

In [16]:
# Compute gini
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [6]:
def gini(solution, submission):
    df = zip(solution, submission, range(len(solution)))
    df = sorted(df, key=lambda x: (x[1],-x[2]), reverse=True)
    rand = [float(i+1)/float(len(df)) for i in range(len(df))]
    totalPos = float(sum([x[0] for x in df]))
    cumPosFound = [df[0][0]]
    for i in range(1,len(df)):
        cumPosFound.append(cumPosFound[len(cumPosFound)-1] + df[i][0])
    Lorentz = [float(x)/totalPos for x in cumPosFound]
    Gini = [Lorentz[i]-rand[i] for i in range(len(df))]
    return sum(Gini)

def normalized_gini(solution, submission):
    normalized_gini = gini(solution, submission)/gini(solution, solution)
    return normalized_gini

In [20]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    #assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)['target'].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': 'target', 'target': 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': 'target', 'target': 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
    #'ps_reg_F',
    #'ps_reg_M'
]
# add combinations
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

# Read data
train_df = pd.read_csv('../data/train.csv', na_values="-1") # .iloc[0:200,:]
test_df = pd.read_csv('../data/test.csv', na_values="-1")
print('imported data')

# Process data
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

print('begining feature engineering')
start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('\n\tCurrent feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)
    
X = train_df[train_features]
test_df = test_df[train_features]

print('\nDone adding features, smoothing categorical featuers now.. ')
#target categoricals 
f_cats = [f for f in X.columns if "_cat" in f]

for f in f_cats:
    X[f + "_avg"], test_df[f + "_avg"] = target_encode(trn_series=X[f],
                                         tst_series=test_df[f],
                                         target=train_df['target'],
                                         min_samples_leaf=200,
                                         smoothing=10,
                                         noise_level=0)
    
X = X.fillna(-1)
test_df = test_df.fillna(-1)
y_valid_pred = 0*y
y_test_pred = 0

print('Done ... I can start learning now.. ')

imported data
begining feature engineering

	Current feature                                 ps_reg_01_plus_ps_car_02_cat    1 in   0.0
	Current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.0
Done adding features, smoothing categorical featuers now.. 


/home/laith/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Done ... I can start learning now.. 


In [59]:
x_train = X.values
y_train = y.values
x_test = test_df.values

In [62]:
clf = ExtraTreesClassifier(n_jobs=-1)

In [64]:
# A list of dictionaries to specify the parameters we want to tune.
param_grid = { "n_estimators" : [50,25,10,60],
               'criterion': ['gini'],
               'max_features': [2,3,4,5,6,7,'auto'],
               "max_depth" :  [i for i in range(1,30,1)],
               'min_samples_split': [2,3,4,5,6,7,8],
               'max_leaf_nodes': [None,2,3,4,8,10,15],
               'min_impurity_decrease':[0,0.01,0.05,0.1,0.15,0.2],
               'min_weight_fraction_leaf':[0,0.01,0.05,0.1,0.15,0.2],
               "min_samples_leaf" : [1,2,3,4,5,6,7,8],
               'bootstrap':[True, False]
               }

n_iter_search = 200
#gini_scorer = make_scorer(eval_gini, greater_is_better = True)
scoring = {'AUC': 'roc_auc'}#, 'gini': gini_scorer}
random_search = RandomizedSearchCV(clf, 
                                   param_distributions=param_grid,
                                   scoring = scoring,
                                   n_iter=n_iter_search,
                                   n_jobs=-1,
                                   error_score=0,
                                   refit='AUC',
                                   cv= KFold(n_splits=5, shuffle=True),
                                   verbose=2)
random_search.fit(x_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV] n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=8, min_samples_leaf=7, min_impurity_decrease=0.2, max_leaf_nodes=None, max_features=7, max_depth=10, criterion=gini, bootstrap=False 
[CV] n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=8, min_samples_leaf=7, min_impurity_decrease=0.2, max_leaf_nodes=None, max_features=7, max_depth=10, criterion=gini, bootstrap=False 
[CV] n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=8, min_samples_leaf=7, min_impurity_decrease=0.2, max_leaf_nodes=None, max_features=7, max_depth=10, criterion=gini, bootstrap=False 
[CV] n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=8, min_samples_leaf=7, min_impurity_decrease=0.2, max_leaf_nodes=None, max_features=7, max_depth=10, criterion=gini, bootstrap=False 
[CV]  n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=8, min_samples_leaf=7, min_impurity_decrease

[CV] n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=8, min_samples_leaf=5, min_impurity_decrease=0.15, max_leaf_nodes=3, max_features=3, max_depth=27, criterion=gini, bootstrap=False 
[CV]  n_estimators=60, min_weight_fraction_leaf=0.2, min_samples_split=5, min_samples_leaf=4, min_impurity_decrease=0.15, max_leaf_nodes=15, max_features=6, max_depth=13, criterion=gini, bootstrap=True, total=   9.5s


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   17.9s


[CV]  n_estimators=50, min_weight_fraction_leaf=0, min_samples_split=4, min_samples_leaf=6, min_impurity_decrease=0.15, max_leaf_nodes=3, max_features=6, max_depth=9, criterion=gini, bootstrap=True, total=   7.7s
[CV]  n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=8, min_samples_leaf=5, min_impurity_decrease=0.15, max_leaf_nodes=3, max_features=3, max_depth=27, criterion=gini, bootstrap=False, total=   4.0s
[CV] n_estimators=25, min_weight_fraction_leaf=0, min_samples_split=5, min_samples_leaf=3, min_impurity_decrease=0.1, max_leaf_nodes=10, max_features=auto, max_depth=20, criterion=gini, bootstrap=False 
[CV] n_estimators=25, min_weight_fraction_leaf=0, min_samples_split=5, min_samples_leaf=3, min_impurity_decrease=0.1, max_leaf_nodes=10, max_features=auto, max_depth=20, criterion=gini, bootstrap=False 
[CV]  n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=8, min_samples_leaf=5, min_impurity_decrease=0.15, max_leaf_nodes=3, max_features=3, max_de

[CV]  n_estimators=60, min_weight_fraction_leaf=0.2, min_samples_split=6, min_samples_leaf=7, min_impurity_decrease=0, max_leaf_nodes=15, max_features=7, max_depth=26, criterion=gini, bootstrap=False, total=  16.8s
[CV]  n_estimators=50, min_weight_fraction_leaf=0.05, min_samples_split=5, min_samples_leaf=7, min_impurity_decrease=0.01, max_leaf_nodes=3, max_features=5, max_depth=28, criterion=gini, bootstrap=False, total=  10.3s
[CV] n_estimators=25, min_weight_fraction_leaf=0.1, min_samples_split=3, min_samples_leaf=1, min_impurity_decrease=0.1, max_leaf_nodes=3, max_features=4, max_depth=11, criterion=gini, bootstrap=False 
[CV]  n_estimators=50, min_weight_fraction_leaf=0.05, min_samples_split=5, min_samples_leaf=7, min_impurity_decrease=0.01, max_leaf_nodes=3, max_features=5, max_depth=28, criterion=gini, bootstrap=False, total=   8.8s
[CV] n_estimators=25, min_weight_fraction_leaf=0.1, min_samples_split=3, min_samples_leaf=1, min_impurity_decrease=0.1, max_leaf_nodes=3, max_featur

...

[CV] n_estimators=25, min_weight_fraction_leaf=0, min_samples_split=4, min_samples_leaf=8, min_impurity_decrease=0.1, max_leaf_nodes=10, max_features=7, max_depth=3, criterion=gini, bootstrap=True 
[CV] n_estimators=60, min_weight_fraction_leaf=0.1, min_samples_split=3, min_samples_leaf=3, min_impurity_decrease=0.15, max_leaf_nodes=8, max_features=3, max_depth=6, criterion=gini, bootstrap=False 
[CV] n_estimators=60, min_weight_fraction_leaf=0.1, min_samples_split=3, min_samples_leaf=3, min_impurity_decrease=0.15, max_leaf_nodes=8, max_features=3, max_depth=6, criterion=gini, bootstrap=False 
[CV]  n_estimators=60, min_weight_fraction_leaf=0.2, min_samples_split=5, min_samples_leaf=4, min_impurity_decrease=0.2, max_leaf_nodes=3, max_features=5, max_depth=13, criterion=gini, bootstrap=True, total=   4.5s
[CV] n_estimators=60, min_weight_fraction_leaf=0.1, min_samples_split=3, min_samples_leaf=3, min_impurity_decrease=0.15, max_leaf_nodes=8, max_features=3, max_depth=6, criterion=gini, b

[CV]  n_estimators=50, min_weight_fraction_leaf=0.1, min_samples_split=3, min_samples_leaf=2, min_impurity_decrease=0.1, max_leaf_nodes=2, max_features=7, max_depth=6, criterion=gini, bootstrap=True, total=   7.2s
[CV] n_estimators=25, min_weight_fraction_leaf=0.01, min_samples_split=2, min_samples_leaf=6, min_impurity_decrease=0.05, max_leaf_nodes=None, max_features=auto, max_depth=4, criterion=gini, bootstrap=False 
[CV]  n_estimators=50, min_weight_fraction_leaf=0.1, min_samples_split=3, min_samples_leaf=2, min_impurity_decrease=0.1, max_leaf_nodes=2, max_features=7, max_depth=6, criterion=gini, bootstrap=True, total=   8.6s
[CV] n_estimators=25, min_weight_fraction_leaf=0.01, min_samples_split=2, min_samples_leaf=6, min_impurity_decrease=0.05, max_leaf_nodes=None, max_features=auto, max_depth=4, criterion=gini, bootstrap=False 
[CV] n_estimators=25, min_weight_fraction_leaf=0.01, min_samples_split=2, min_samples_leaf=6, min_impurity_decrease=0.05, max_leaf_nodes=None, max_features=

[CV] n_estimators=10, min_weight_fraction_leaf=0.2, min_samples_split=4, min_samples_leaf=6, min_impurity_decrease=0.05, max_leaf_nodes=8, max_features=auto, max_depth=18, criterion=gini, bootstrap=True 
[CV]  n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=7, min_samples_leaf=6, min_impurity_decrease=0.2, max_leaf_nodes=15, max_features=3, max_depth=5, criterion=gini, bootstrap=False, total=   0.8s
[CV] n_estimators=10, min_weight_fraction_leaf=0.2, min_samples_split=4, min_samples_leaf=6, min_impurity_decrease=0.05, max_leaf_nodes=8, max_features=auto, max_depth=18, criterion=gini, bootstrap=True 
[CV]  n_estimators=10, min_weight_fraction_leaf=0.15, min_samples_split=7, min_samples_leaf=6, min_impurity_decrease=0.2, max_leaf_nodes=15, max_features=3, max_depth=5, criterion=gini, bootstrap=False, total=   0.9s
[CV]  n_estimators=10, min_weight_fraction_leaf=0.2, min_samples_split=4, min_samples_leaf=6, min_impurity_decrease=0.05, max_leaf_nodes=8, max_features=auto,

[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 15.5min finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
          error_score=0,
          estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=200, n_jobs=-1,
          param_distributions={'n_estimators': [50, 25, 10, 60], 'criterion': ['gini'], 'max_features': [2, 3, 4, 5, 6, 7, 'auto'], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8], 'max_leaf_no...01, 0.05, 0.1, 0.15, 0.2], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8], 'bootstrap': [True, False]},
 

In [44]:
random_search.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_estimators', 'param_min_weight_fraction_leaf', 'param_min_samples_split', 'param_min_samples_leaf', 'param_min_impurity_decrease', 'param_max_leaf_nodes', 'param_max_features', 'param_max_depth', 'param_criterion', 'param_bootstrap', 'params', 'split0_test_AUC', 'split1_test_AUC', 'split2_test_AUC', 'split3_test_AUC', 'split4_test_AUC', 'mean_test_AUC', 'std_test_AUC', 'rank_test_AUC', 'split0_train_AUC', 'split1_train_AUC', 'split2_train_AUC', 'split3_train_AUC', 'split4_train_AUC', 'mean_train_AUC', 'std_train_AUC'])

In [65]:
random_search.best_score_

0.61994830824873115

In [67]:
random_search.best_params_

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 6,
 'max_features': 2,
 'max_leaf_nodes': 8,
 'min_impurity_decrease': 0,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'min_weight_fraction_leaf': 0,
 'n_estimators': 60}

In [47]:
random_search.cv_results_

{'mean_fit_time': array([ 1.02662992,  1.37420678]),
 'mean_score_time': array([ 0.18260417,  0.15462785]),
 'mean_test_AUC': array([ 0.50085979,  0.49981098]),
 'mean_train_AUC': array([ 0.50015122,  0.50001164]),
 'param_bootstrap': masked_array(data = [True True],
              mask = [False False],
        fill_value = ?),
 'param_criterion': masked_array(data = ['entropy' 'entropy'],
              mask = [False False],
        fill_value = ?),
 'param_max_depth': masked_array(data = [12 1],
              mask = [False False],
        fill_value = ?),
 'param_max_features': masked_array(data = [4 7],
              mask = [False False],
        fill_value = ?),
 'param_max_leaf_nodes': masked_array(data = [8 10],
              mask = [False False],
        fill_value = ?),
 'param_min_impurity_decrease': masked_array(data = [0.66 0.76],
              mask = [False False],
        fill_value = ?),
 'param_min_samples_leaf': masked_array(data = [0.22 0.42],
              mask = [False